#### This notebook, as the name suggests, is just for fun. It holds any interesting findings or any "aha" moments.

<b>1.</b> In the cells below you will see that the number of non-zeros in the matrix read from .mtx file is much larger than the number of lines (hence the number of edges) in the graph. What gives? 

Hint: See the ratio of count of non-zeros and number of edges! 

In [9]:
from scipy import io
import numpy as np

mtx_path = '..\data\socfb-American75.mtx'
mat = io.mmread(mtx_path)

print("number of vertices:", mat.shape[0])
print("percentage of non-zero elements in the matrix:", np.round(mat.count_nonzero()/np.prod(mat.shape)*100, 2))

number of vertices: 6386
percentage of non-zero elements in the matrix: 1.07


In [8]:
with open('../data/socfb-American75.edgelist') as file:
    lines = file.readlines()

print("number of edges in edgelist file =", len(lines), ", number of non-zero entries in matrix =", mat.count_nonzero())
print("Ratio of non-zero entries to number of edges =", mat.count_nonzero()/len(lines))

number of edges in edgelist file = 217662 , number of non-zero entries in matrix = 435324
Ratio of non-zero entries to number of edges = 2.0


#### Reason: The .mtx file with symmetric format stores only the entries on or below the main diagonal. So if you have edge from 5 to 1, same edge would be present in entry 1-5 but would not be mentioned in the mtx file to save space.